In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *

In [3]:
def create_model(n_inputs, n_units):
    word_inputs = Input(shape=(n_inputs, n_units), name="word_inputs")
    y = LSTM(n_units)(word_inputs)
    model = Model(word_inputs, y)
    return model

In [30]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)

In [36]:
seq_embeddings = np.load('results/char_embedding.npy')
embed_size = seq_embeddings.shape[1]
batch_size = 100
n_seq = 20
steps_per_batch = len(words)//batch_size

In [37]:
def generate(data, embeds, word2int, batch_size, n_seq):
    embed_size = embeds.shape[1]
    ci = 0  # current_word_index
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, n_seq, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        for bi in range(batch_size):
            context, ci = get_context_words(data, ci, n_seq + 1)
            seq_vecs = embeds[context[:-1]]
            batch_inputs[bi] = seq_vecs
            batch_labels[bi] = embeds[context[-1]]
            ci = ci + 1
            
        yield batch_inputs, batch_labels


In [41]:
gen = generate(int_words, seq_embeddings, word2int, batch_size, n_seq)

In [42]:
model = create_model(n_seq, embed_size)
adam = keras.optimizers.Nadam(0.0001)
model.compile(optimizer=adam, loss='mse')

In [43]:
history = model.fit_generator(gen, steps_per_epoch=steps_per_batch, epochs = 2)

Epoch 1/2
 2060/40894 [>.............................] - ETA: 30:19 - loss: 28.7631

KeyboardInterrupt: 